In [27]:
import nltk
from nltk.stem.lancaster import LancasterStemmer 
stemmer = LancasterStemmer() 
import numpy as np 
import tensorflow as tf 
import random 
import json  
with open('intents.json') as json_data:     intents = json.load(json_data)

In [28]:
# How documents looks like
[(['Hi'], 'greeting'),
  (['How', 'are', 'you'], 'greeting'),
  (['Is', 'anyone', 'there', '?'], 'greeting'),
  (['Hello'], 'greeting'),
  (['Good', 'day'], 'greeting')],
# tags or classes
['stories', 'weather', 'google', 'wikipedia', 'news'],
#dictionary sample
(['updates',
  '?',
  'todays',
  'news',
  'top',
  'headlines',
  'current',
  'news',
  'news',
  'headlines'])
words = []
classes = []
documents = []
ignore_words = ['?','you','the', 'a', 'an','do','what','where','why','to','from','if','which' ]
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# remove duplicates
classes = sorted(list(set(classes)))

In [29]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
# output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-29-5d9aa7aa2ce8>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
input_layer = Input(shape=(len(train_x[0])))
layer1 = Dense(100,activation='relu')(input_layer)
layer2 = Dense(50,activation='relu')(layer1)
output = Dense(len(train_y[0]),activation='sigmoid')(layer2)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
model.summary()
model.compile(optimizer="Adam", loss="mse", metrics=['accuracy'])
model.fit(train_x, train_y, epochs=30, batch_size=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 80)]              0         
                                                                 
 dense_3 (Dense)             (None, 100)               8100      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 9)                 459       
                                                                 
Total params: 13,609
Trainable params: 13,609
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - 1s 780ms/step - loss: 0.2411 - accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 11ms/step - loss: 0.2364 - accur

In [40]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

# create a data structure to hold user context
context = {}


ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words).reshape((1,-1))])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        val = random.choice(i['responses'])
                        #speech = Speech(val, lang)
                        #sox_effects = ("speed", "1.0")
                        #speech.play(sox_effects)
                        # a random response from the intent
                        return print(val)

            results.pop(0)

ques = input("hey!! Ask me something -: ")
print(classify(ques))
response(ques)

hey!! Ask me something -: hi


TypeError: 'dict' object is not callable